In [28]:
import pymongo 
import pandas as pd

In [4]:
conn_str = "mongodb://localhost:27017"
client = pymongo.MongoClient(conn_str,serverSelectionTimeoutMS=5000)
try:
    print(client.server_info())
except Exception:
    print("No conectado")

{'version': '6.0.2', 'gitVersion': '94fb7dfc8b974f1f5343e7ea394d0d9deedba50e', 'targetMinOS': 'Windows 7/Windows Server 2008 R2', 'modules': [], 'allocator': 'tcmalloc', 'javascriptEngine': 'mozjs', 'sysInfo': 'deprecated', 'versionArray': [6, 0, 2, 0], 'openssl': {'running': 'Windows SChannel'}, 'buildEnvironment': {'distmod': 'windows', 'distarch': 'x86_64', 'cc': 'cl: Microsoft (R) C/C++ Optimizing Compiler Version 19.31.31107 for x64', 'ccflags': '/nologo /WX /FImongo/platform/basic.h /fp:strict /EHsc /W3 /wd4068 /wd4244 /wd4267 /wd4290 /wd4351 /wd4355 /wd4373 /wd4800 /wd4251 /wd4291 /we4013 /we4099 /we4930 /errorReport:none /MD /O2 /Oy- /bigobj /utf-8 /permissive- /Zc:__cplusplus /Zc:sizedDealloc /volatile:iso /diagnostics:caret /std:c++17 /Gw /Gy /Zc:inline', 'cxx': 'cl: Microsoft (R) C/C++ Optimizing Compiler Version 19.31.31107 for x64', 'cxxflags': '/TP', 'linkflags': '/nologo /DEBUG /INCREMENTAL:NO /LARGEADDRESSAWARE /OPT:REF', 'target_arch': 'x86_64', 'target_os': 'windows',

In [56]:
db = client["actMongoDb"]
clics = db["Clics"]
tweets=db["Tweets"]

In [27]:
"""Seleccionar las columnas “_id”, “a”, y “cy” de clics
    Ordenar por cy en orden descendente
    Obtener sólo 3 filas
    Omitir la primera
""" 

qry = clics.find({},{"a":1,"cy":1}).sort("cy").limit(3)

for x in qry:
    print(x)

{'_id': ObjectId('6386954e1a5173884db15ed4'), 'a': 'UCWEB/2.0 (Linux; U; Adr 4.1.1; en-US; GT-P3100) U2/1.0.0 UCBrowser/8.2.0.242 U2/1.0.0 Mobile'}
{'_id': ObjectId('6386954e1a5173884db15ed1'), 'a': 'Opera/9.80 (VRE; Opera Mini/4.2/29.3594; U; en) Presto/2.8.119 Version/11.10'}
{'_id': ObjectId('6386954e1a5173884db15ecf')}


In [35]:
"""Crear una serie con todos los valores únicos de “a”."""
cursor_a = clics.find({},{"a":1}).distinct("a")
serie_a = pd.Series(list(cursor_a))


0      0 Opera/9.80 (J2ME/MIDP; Opera Mini/6.5.27891/...
1      BlackBerry8520/5.0.0.1075 Profile/MIDP-2.1 Con...
2      BlackBerry8520/5.0.0.592 Profile/MIDP-2.1 Conf...
3      BlackBerry8520/5.0.0.592 Profile/MIDP-2.1 Conf...
4      BlackBerry8900/5.0.0.1036 Profile/MIDP-2.1 Con...
                             ...                        
554                               Xenu Link Sleuth/1.3.8
555    Yes Streamer 1.5.7.2 Indexer (powered by TVers...
556    findlinks/2.6 (+http://wortschatz.uni-leipzig....
557    iCoreService (unknown version) CFNetwork/520.5...
558    ichiro/3.0 (http://search.goo.ne.jp/option/use...
Length: 559, dtype: object


In [36]:
""""Contar los usuarios de Opera y Mozilla utilizando el campo "a"
Crear una serie a partir de los resultados
"""

Usr_Mozilla_Opera = pd.DataFrame(clics.aggregate([
    {
        "$match":{
            "$or":[ {"a":{"$regex":"Opera"}}
                   ,{"a":{"$regex":"Mozilla"}}
                  ]
        }
    },
    {"$count":"a"}
]))

Usr_Mozilla_Opera

,a
0,2830


In [52]:
'''Encontrar cuantos usuarios hablan inglés'''

usr_ingles = clics.count_documents({"al":{"$regex":"en"}}) 

usr_ingles

1278

In [54]:
'''Encontrar cuántos usuarios son de Seoul.'''

usr_seoul = clics.count_documents({"cy":{"$regex":"Seoul"}}) 
usr_seoul

6

In [59]:
'''Tweets'''
'''¿Cuántas filas contiene la collección?'''
tweets.count_documents({})

122

In [60]:
'''Enumerar los screen_names de los usuarios que han tuiteado más de una vez'''
pd.DataFrame(tweets.aggregate(
    [ 
        { "$group":  { "_id": "$user.screen_name", "conteo": { "$sum": 1 } } },
        {"$match": {"conteo": {"$gt": 1}}} 
    ]
))



,_id,conteo
0,SoyLatte,2
1,arianaskittles,2


In [61]:
'''¿Cuántos tweets son sobre Starbucks?'''
tweets.count_documents({"text":{"$regex":"Starbucks"}})

10

In [62]:
'''Enumerar los tweets en español'''
tweets.count_documents({"lang":{"$regex":"es"}})

17